# spring核心以及原理
* spring架构图：<img src='../images/javaee/spring结构图.png' width='350px'>
* springMVC接受请求流程：
    * 发现请求Locale
        * localeResolver
    * 如果是多媒体请求，提交给多媒体请求处理
        * multipartResolver
    * 定位Controller处理器
        * handlerMapping，基于最长匹配模式路径
        * 通过HandlerExecutionChain方式完成处理，该chain中包含了intercept和controller
        * AbstractController：检测HTTP请求方法(GET, POST)，检测是否请求有session或cookie，判断头部缓存标志，进行会话同步，通过handleRequestInternal调用子类方法处理请求
        * DataBinder，PropertyEditor将请求参数绑定到对应的bean对象字段中，每个不同的对象都有对应的PropertyEditor，BaseCommandController实现该调度
        * SimpleFormController表单请求简单实现，可以将表单数据绑定到对应的model，并且处理sumbit等事件
        * MethodNameResolver当一个Controller中有多个方法处理Action时，该类的子类负责解析URL到方法的映射
    * 定位所有的拦截器
        * handlerAdapter
        * handlerInterceptor
        * messageConvert在拦截器中定义
    * 调用拦截器的preHandle()方法
    * 调用Controller中的处理函数
    * 调用拦截器中的postHandle()方法
    * 如有异常，用HandlerExceptionResolver处理
        * handlerExceptionResolver
    * 如果无异常，返回ModalAndView，通过视图渲染器完成渲染
        * viewResolver，将model绑定到视图模板中（JSP, FreeMarker, Velocity），动态生成前端web页面给用户
    * 调用拦截器上的afterCompletion()方法
* DispatcherServlet
    * ContextLoaderListener加载web.xml配置文件，相当于ServletContextListener，会初始化WebApplicationContext，并存储到ServletContext中
    * 初始化过程：将servlet转化为BeanWrapper类型，将变量及参数注入到Spring的Bean中，也就是将初始化自己所写的bean，并对bean进行参数注入
        * 验证bean参数，以及初始化bean参数
        * 将该bean转成beanwrapper实例
        * 注册相对于Resource的属性编辑器
        * 属性注入
        * servletBean初始化，初始化WebApplicationContext，会重新刷新所有的bean配置
            * 初始化multipartResolver
            * 初始化localeResolver
            * 初始化themeResolver
            * 初始化handlerMapping，所有的请求是handlerMapping根据WebApplicationContext的参数匹配出来的HandlerExecutionChain
            * 初始化handlerAdapter，集成第三方框架的适配器，执行第三方框架的相关功能，将自己的controller传给adapter，在adapter中处理真正的请求，查找真正的handler处理请求
            * 初始化handlerExceptionResolver
            * 初始化requestToViewNameTranslator，在没有显示返回view的名字时，按约定解析url生成view的名字
            * 初始化viewResolver
            * 初始化flashMapManager
    * 处理逻辑：
        * 所有请求都是在doService()中处理的
        * 判断请求是否是多媒体请求，使用multipartResolver处理该类请求
        * 根据url在handlerMapping查找对应的controller，并生成HandlerExecutionChain
        * 将拦截器加入到HandlerExecutionChain中
        * 通过HandlerExecutionChain中的handler查找对应的handlerAdapter，通过handlerAdapter查找真正的handler处理请求
        * 处理缓存
        * 拦截器开始处理逻辑
        * adapter开始处理逻辑，本质是执行handler的逻辑
        * handlerExceptionResolver处理异常
        * viewResolver完成视图渲染和跳转